<a href="https://colab.research.google.com/github/sonu2012/Information-Retrieval/blob/master/Information_Retrieval_SONU_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None
from google.colab import drive
drive.mount('/content/drive')
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/NLP/archive'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        with open(os.path.join(dirname, filename)) as f:
            line_count = 0
            id_set = set()
            for l in f.read():
                line_count += 1
                if filename == "CISI.REL":
                    id_set.add(l.lstrip(" ").split(" ")[0])
                elif l.startswith(".I "):
                    id_set.add(l.split(" ")[1].strip())
            print(f"{filename} : {len(id_set)} items, over {line_count} lines.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NLP/archive/CISI.ALL
CISI.ALL : 0 items, over 2119351 lines.
/content/drive/MyDrive/NLP/archive/CISI.QRY
CISI.QRY : 0 items, over 66632 lines.
/content/drive/MyDrive/NLP/archive/CISI.REL
CISI.REL : 14 items, over 77850 lines.
/content/drive/MyDrive/NLP/archive/featurematrix.csv
featurematrix.csv : 0 items, over 10368869 lines.
/content/drive/MyDrive/NLP/archive/featurematrixlemm.csv
featurematrixlemm.csv : 0 items, over 6801692 lines.
/content/drive/MyDrive/NLP/archive/doc_sw_results.csv
doc_sw_results.csv : 0 items, over 3326479 lines.
/content/drive/MyDrive/NLP/archive/dfdoc_mesh_tfidf.csv
dfdoc_mesh_tfidf.csv : 0 items, over 64310560 lines.
/content/drive/MyDrive/NLP/archive/qrdoc_mesh_tfidf.csv
qrdoc_mesh_tfidf.csv : 0 items, over 4920877 lines.
/content/drive/MyDrive/NLP/archive/doc_sw_cosine.csv
doc_sw_cosine.csv : 0 items, over 2

In [3]:
with open('/content/drive/MyDrive/NLP/archive/CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
# print n lines
n = 10
for l in lines[:n]:
    print(l)

.I 1
.T 18 Editions of the Dewey Decimal Classifications
.A Comaromi, J.P.
.W The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad.
.X 1	5	1 92	1	1 262	1	1 556	1	1 1004	1	1 1024	1	1 1024	1	1
.I 2
.T Use Made of Technical Libraries
.A Slater, M.
.W This report is an analysis of 6300 acts of use in 104 technical libraries in the United Kingdom. Library use is only one aspect of the wider pattern of information use.  Information transfer in libraries is restricted to the use of documents.  It takes no account of doc

In [4]:
doc_set = {}
doc_id = ""
doc_text = ""
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip()
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.

# Print something to see the dictionary structure, etc.
print(f"Number of documents = {len(doc_set)}" + ".\n")
print(doc_set["5"]) # note that the dictionary indexes are strings, not numbers. 
        

Number of documents = 1460.

A Library Management Game: a report on a research project Brophy, P. Although the use of games in professional education has become widespread only during the last decade, the method has been used in a number of fields for many hundreds of years. Its origins have been traced to simple war games, used in military training when the "real thing" was either unavailable or too dangerous.  In more recent times, these games have become more and more sophisticated, and many now use large electronic computers to handle the complex calculations involved. Since 1956, when the first well-developed management game was introduced, the technique has spread rapidly into a wide variety of disciplines and today it is used at all levels of education, from primary school classes to courses for experienced professional men and women.  One of the main causes of this "game explosion" has been the rapid development of sophisticated management techniques, such as simulation and mat

In [5]:
with open('/content/drive/MyDrive/NLP/archive/CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = l.split(" ")[1].strip()
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""
    
# Print something to see the dictionary structure, etc.
print(f"Number of queries = {len(qry_set)}" + ".\n")
print(qry_set["3"]) # note that the dictionary indexes are strings, not numbers.

Number of queries = 112.

What is information science?  Give definitions where possible.


In [6]:
rel_set = {}
with open('/content/drive/MyDrive/NLP/archive/CISI.REL') as f:
    for l in f.readlines():
        qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]
        doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)
        if qry_id == "7":
            print(l.strip("\n"))
    
# Print something to see the dictionary structure, etc.
print(f"\nNumber of mappings = {len(rel_set)}" + ".\n")
print(rel_set["7"]) # note that the dictionary indexes are strings, not numbers

     7    310	0	0.000000
     7    320	0	0.000000
     7    332	0	0.000000
     7    375	0	0.000000
     7    376	0	0.000000
     7    645	0	0.000000
     7    724	0	0.000000
     7    725	0	0.000000

Number of mappings = 76.

['310', '320', '332', '375', '376', '645', '724', '725']


In [7]:
docs = []
for i in range(1,len(doc_set)+1):
    docs.append(doc_set[str(i)])
    dfdoc=pd.DataFrame(docs,columns={'Docs'})

In [8]:
print(dfdoc)

                                                   Docs
0     18 Editions of the Dewey Decimal Classificatio...
1     Use Made of Technical Libraries Slater, M. Thi...
2     Two Kinds of Power An Essay on Bibliographic C...
3     Systems Analysis of a University Library; fina...
4     A Library Management Game: a report on a resea...
...                                                 ...
1455  World Dynamics Forrester, J.W. Over the last s...
1456  World Trends in Library Education Bramley, G. ...
1457  Legal Restrictions on Exploitation of the Pate...
1458  Language and Thought Poluskin, V.A. This book ...
1459  Modern Integral Information Systems for Chemis...

[1460 rows x 1 columns]


In [9]:
dfdoc["text_lower"] = dfdoc["Docs"].str.lower()
dfdoc.head()

,Docs,text_lower
0,18 Editions of the Dewey Decimal Classificatio...,18 editions of the dewey decimal classificatio...
1,"Use Made of Technical Libraries Slater, M. Thi...","use made of technical libraries slater, m. thi..."
2,Two Kinds of Power An Essay on Bibliographic C...,two kinds of power an essay on bibliographic c...
3,Systems Analysis of a University Library; fina...,systems analysis of a university library; fina...
4,A Library Management Game: a report on a resea...,a library management game: a report on a resea...


In [10]:
# drop the new column created in last cell
dfdoc.drop(["text_lower"], axis=1, inplace=True)

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

dfdoc["text_wo_punct"] = dfdoc["Docs"].apply(lambda text: remove_punctuation(text))
dfdoc.head()

,Docs,text_wo_punct
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions of the Dewey Decimal Classificatio...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made of Technical Libraries Slater M This ...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds of Power An Essay on Bibliographic C...
3,Systems Analysis of a University Library; fina...,Systems Analysis of a University Library final...
4,A Library Management Game: a report on a resea...,A Library Management Game a report on a resear...


In [11]:
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [12]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

dfdoc["text_wo_stop"] = dfdoc["text_wo_punct"].apply(lambda text: remove_stopwords(text))
dfdoc.head()

,Docs,text_wo_punct,text_wo_stop
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made of Technical Libraries Slater M This ...,Use Made Technical Libraries Slater M This rep...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...
3,Systems Analysis of a University Library; fina...,Systems Analysis of a University Library final...,Systems Analysis University Library final repo...
4,A Library Management Game: a report on a resea...,A Library Management Game a report on a resear...,A Library Management Game report research proj...


In [13]:
dfdoc["text_to_lower"] = dfdoc["text_wo_stop"].str.lower()
dfdoc.head(10)

,Docs,text_wo_punct,text_wo_stop,text_to_lower
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...,18 editions dewey decimal classifications coma...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made of Technical Libraries Slater M This ...,Use Made Technical Libraries Slater M This rep...,use made technical libraries slater m this rep...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...,two kinds power an essay bibliographic control...
3,Systems Analysis of a University Library; fina...,Systems Analysis of a University Library final...,Systems Analysis University Library final repo...,systems analysis university library final repo...
4,A Library Management Game: a report on a resea...,A Library Management Game a report on a resear...,A Library Management Game report research proj...,a library management game report research proj...
5,"Abstracting Concepts and Methods Borko, H. Gra...",Abstracting Concepts and Methods Borko H Gradu...,Abstracting Concepts Methods Borko H Graduate ...,abstracting concepts methods borko h graduate ...
6,Academic Library Buildings A Guide to Architec...,Academic Library Buildings A Guide to Architec...,Academic Library Buildings A Guide Architectur...,academic library buildings a guide architectur...
7,The Academic Library Essays in Honor of Guy R....,The Academic Library Essays in Honor of Guy R ...,The Academic Library Essays Honor Guy R Lyle F...,the academic library essays honor guy r lyle f...
8,"Access to Libraries in College Hyman, R.T. Thi...",Access to Libraries in College Hyman RT This s...,Access Libraries College Hyman RT This study a...,access libraries college hyman rt this study a...
9,"Access to Periodical Resources Palmour, V.E. T...",Access to Periodical Resources Palmour VE The ...,Access Periodical Resources Palmour VE The pur...,access periodical resources palmour ve the pur...


In [14]:
from collections import Counter
cnt = Counter()
for text in dfdoc["text_to_lower"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('the', 2090),
 ('information', 1573),
 ('library', 1207),
 ('system', 751),
 ('a', 727),
 ('libraries', 611),
 ('data', 586),
 ('research', 565),
 ('retrieval', 550),
 ('this', 534)]

In [15]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

dfdoc["text_wo_stopfreq"] = dfdoc["text_to_lower"].apply(lambda text: remove_freqwords(text))
dfdoc.head()

,Docs,text_wo_punct,text_wo_stop,text_to_lower,text_wo_stopfreq
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...,18 editions dewey decimal classifications coma...,18 editions dewey decimal classifications coma...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made of Technical Libraries Slater M This ...,Use Made Technical Libraries Slater M This rep...,use made technical libraries slater m this rep...,use made technical slater m report analysis 63...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...,two kinds power an essay bibliographic control...,two kinds power an essay bibliographic control...
3,Systems Analysis of a University Library; fina...,Systems Analysis of a University Library final...,Systems Analysis University Library final repo...,systems analysis university library final repo...,systems analysis university final report proje...
4,A Library Management Game: a report on a resea...,A Library Management Game a report on a resear...,A Library Management Game report research proj...,a library management game report research proj...,management game report project brophy p althou...


In [16]:
# Drop the two columns which are no more needed 
dfdoc.drop(["text_wo_punct", "text_to_lower"], axis=1, inplace=True)

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

dfdoc["text_wo_stopfreqrare"] = dfdoc["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
dfdoc.head()

,Docs,text_wo_stop,text_wo_stopfreq,text_wo_stopfreqrare
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...,18 editions dewey decimal classifications coma...,18 editions dewey decimal classifications coma...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made Technical Libraries Slater M This rep...,use made technical slater m report analysis 63...,use made technical slater m report analysis 63...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...,two kinds power an essay bibliographic control...,two kinds power an essay bibliographic control...
3,Systems Analysis of a University Library; fina...,Systems Analysis University Library final repo...,systems analysis university final report proje...,systems analysis university final report proje...
4,A Library Management Game: a report on a resea...,A Library Management Game report research proj...,management game report project brophy p althou...,management game report project brophy p althou...


In [17]:
from nltk.stem.porter import PorterStemmer

# Drop the two columns 
dfdoc.drop(["text_wo_stopfreq"], axis=1, inplace=True) 

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

dfdoc["text_stemmed"] = dfdoc["text_wo_stopfreqrare"].apply(lambda text: stem_words(text))
dfdoc.head()

,Docs,text_wo_stop,text_wo_stopfreqrare,text_stemmed
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...,18 editions dewey decimal classifications coma...,18 edit dewey decim classif comaromi jp presen...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made Technical Libraries Slater M This rep...,use made technical slater m report analysis 63...,use made technic slater m report analysi 6300 ...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...,two kinds power an essay bibliographic control...,two kind power an essay bibliograph control wi...
3,Systems Analysis of a University Library; fina...,Systems Analysis University Library final repo...,systems analysis university final report proje...,system analysi univers final report project bu...
4,A Library Management Game: a report on a resea...,A Library Management Game report research proj...,management game report project brophy p althou...,manag game report project brophi p although us...


In [18]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

dfdoc["text_lemmatized"] = dfdoc["text_wo_stopfreqrare"].apply(lambda text: lemmatize_words(text))
dfdoc.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Docs,text_wo_stop,text_wo_stopfreqrare,text_stemmed,text_lemmatized
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...,18 editions dewey decimal classifications coma...,18 edit dewey decim classif comaromi jp presen...,18 edition dewey decimal classification comaro...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made Technical Libraries Slater M This rep...,use made technical slater m report analysis 63...,use made technic slater m report analysi 6300 ...,use made technical slater m report analysis 63...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...,two kinds power an essay bibliographic control...,two kind power an essay bibliograph control wi...,two kind power an essay bibliographic control ...
3,Systems Analysis of a University Library; fina...,Systems Analysis University Library final repo...,systems analysis university final report proje...,system analysi univers final report project bu...,system analysis university final report projec...
4,A Library Management Game: a report on a resea...,A Library Management Game report research proj...,management game report project brophy p althou...,manag game report project brophi p although us...,management game report project brophy p althou...


In [19]:
dfdoc_mesh=pd.DataFrame(dfdoc["text_lemmatized"])

In [20]:
dfdoc_mesh

,text_lemmatized
0,18 edition dewey decimal classification comaro...
1,use made technical slater m report analysis 63...
2,two kind power an essay bibliographic control ...
3,system analysis university final report projec...
4,management game report project brophy p althou...
...,...
1455,world dynamic forrester jw over last several d...
1456,world trend education bramley g one significan...
1457,legal restriction exploitation patent monopoly...
1458,language thought va book considers basic aspec...


In [21]:
def splitbagorwords(text):
  document =text
  bagofwords=document.split(' ')
  return bagofwords

In [22]:
dfdoc_mesh['bagG']=dfdoc_mesh['text_lemmatized'].apply(lambda row: splitbagorwords(row))

In [23]:
dfdoc_mesh

,text_lemmatized,bagG
0,18 edition dewey decimal classification comaro...,"[18, edition, dewey, decimal, classification, ..."
1,use made technical slater m report analysis 63...,"[use, made, technical, slater, m, report, anal..."
2,two kind power an essay bibliographic control ...,"[two, kind, power, an, essay, bibliographic, c..."
3,system analysis university final report projec...,"[system, analysis, university, final, report, ..."
4,management game report project brophy p althou...,"[management, game, report, project, brophy, p,..."
...,...,...
1455,world dynamic forrester jw over last several d...,"[world, dynamic, forrester, jw, over, last, se..."
1456,world trend education bramley g one significan...,"[world, trend, education, bramley, g, one, sig..."
1457,legal restriction exploitation patent monopoly...,"[legal, restriction, exploitation, patent, mon..."
1458,language thought va book considers basic aspec...,"[language, thought, va, book, considers, basic..."


In [24]:
def uniq(text1):
  return set(text1)

In [25]:
dfdoc_mesh['uniquewordscc']=dfdoc_mesh['bagG'].apply(lambda row: uniq(row))

In [26]:
dfdoc_mesh

,text_lemmatized,bagG,uniquewordscc
0,18 edition dewey decimal classification comaro...,"[18, edition, dewey, decimal, classification, ...","{provide, healthy, dewey, continue, 1876, life..."
1,use made technical slater m report analysis 63...,"[use, made, technical, slater, m, report, anal...","{outside, pattern, current, contact, organizat..."
2,two kind power an essay bibliographic control ...,"[two, kind, power, an, essay, bibliographic, c...","{obtain, organization, enter, slogan, mankind,..."
3,system analysis university final report projec...,"[system, analysis, university, final, report, ...","{remained, lack, service, endorsed, 1960s, aca..."
4,management game report project brophy p althou...,"[management, game, report, project, brophy, p,...","{complex, introduced, woman, war, professional..."
...,...,...,...
1455,world dynamic forrester jw over last several d...,"[world, dynamic, forrester, jw, over, last, se...","{believe, interaction, organization, forrester..."
1456,world trend education bramley g one significan...,"[world, trend, education, bramley, g, one, sig...","{current, pattern, world, factor, concentratio..."
1457,legal restriction exploitation patent monopoly...,"[legal, restriction, exploitation, patent, mon...","{brief, invention, legal, respectivediscoverie..."
1458,language thought va book considers basic aspec...,"[language, thought, va, book, considers, basic...","{interaction, complex, language, historical, e..."


In [27]:
dfdoc_mesh["UniwordCount"]=dfdoc_mesh['uniquewordscc'].apply(lambda x: len(str(x).split()))
dfdoc_mesh

,text_lemmatized,bagG,uniquewordscc,UniwordCount
0,18 edition dewey decimal classification comaro...,"[18, edition, dewey, decimal, classification, ...","{provide, healthy, dewey, continue, 1876, life...",43
1,use made technical slater m report analysis 63...,"[use, made, technical, slater, m, report, anal...","{outside, pattern, current, contact, organizat...",60
2,two kind power an essay bibliographic control ...,"[two, kind, power, an, essay, bibliographic, c...","{obtain, organization, enter, slogan, mankind,...",43
3,system analysis university final report projec...,"[system, analysis, university, final, report, ...","{remained, lack, service, endorsed, 1960s, aca...",48
4,management game report project brophy p althou...,"[management, game, report, project, brophy, p,...","{complex, introduced, woman, war, professional...",77
...,...,...,...,...
1455,world dynamic forrester jw over last several d...,"[world, dynamic, forrester, jw, over, last, se...","{believe, interaction, organization, forrester...",73
1456,world trend education bramley g one significan...,"[world, trend, education, bramley, g, one, sig...","{current, pattern, world, factor, concentratio...",64
1457,legal restriction exploitation patent monopoly...,"[legal, restriction, exploitation, patent, mon...","{brief, invention, legal, respectivediscoverie...",74
1458,language thought va book considers basic aspec...,"[language, thought, va, book, considers, basic...","{interaction, complex, language, historical, e...",24


In [28]:
def numwords(uniqtext,bagw):
  numOfWords = dict.fromkeys(uniqtext, 0)
  for word in bagw:
    numOfWords[word] += 1
  return numOfWords

In [29]:
dfdoc_mesh["numwords"]=dfdoc_mesh.apply(lambda row:numwords(row['uniquewordscc'],row['bagG']),axis=1)

In [30]:
dfdoc_mesh

,text_lemmatized,bagG,uniquewordscc,UniwordCount,numwords
0,18 edition dewey decimal classification comaro...,"[18, edition, dewey, decimal, classification, ...","{provide, healthy, dewey, continue, 1876, life...",43,"{'provide': 1, 'healthy': 1, 'dewey': 3, 'cont..."
1,use made technical slater m report analysis 63...,"[use, made, technical, slater, m, report, anal...","{outside, pattern, current, contact, organizat...",60,"{'outside': 1, 'pattern': 1, 'current': 1, 'co..."
2,two kind power an essay bibliographic control ...,"[two, kind, power, an, essay, bibliographic, c...","{obtain, organization, enter, slogan, mankind,...",43,"{'obtain': 1, 'organization': 2, 'enter': 1, '..."
3,system analysis university final report projec...,"[system, analysis, university, final, report, ...","{remained, lack, service, endorsed, 1960s, aca...",48,"{'remained': 1, 'lack': 1, 'service': 1, 'endo..."
4,management game report project brophy p althou...,"[management, game, report, project, brophy, p,...","{complex, introduced, woman, war, professional...",77,"{'complex': 1, 'introduced': 1, 'woman': 1, 'w..."
...,...,...,...,...,...
1455,world dynamic forrester jw over last several d...,"[world, dynamic, forrester, jw, over, last, se...","{believe, interaction, organization, forrester...",73,"{'believe': 1, 'interaction': 1, 'organization..."
1456,world trend education bramley g one significan...,"[world, trend, education, bramley, g, one, sig...","{current, pattern, world, factor, concentratio...",64,"{'current': 1, 'pattern': 1, 'world': 2, 'fact..."
1457,legal restriction exploitation patent monopoly...,"[legal, restriction, exploitation, patent, mon...","{brief, invention, legal, respectivediscoverie...",74,"{'brief': 1, 'invention': 2, 'legal': 2, 'resp..."
1458,language thought va book considers basic aspec...,"[language, thought, va, book, considers, basic...","{interaction, complex, language, historical, e...",24,"{'interaction': 1, 'complex': 1, 'language': 2..."


In [31]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [32]:
dfdoc_mesh["tf"]=dfdoc_mesh.apply(lambda row:computeTF(row['numwords'],row['bagG']),axis=1)

In [33]:
dfdoc_mesh.head(10)

,text_lemmatized,bagG,uniquewordscc,UniwordCount,numwords,tf
0,18 edition dewey decimal classification comaro...,"[18, edition, dewey, decimal, classification, ...","{provide, healthy, dewey, continue, 1876, life...",43,"{'provide': 1, 'healthy': 1, 'dewey': 3, 'cont...","{'provide': 0.018867924528301886, 'healthy': 0..."
1,use made technical slater m report analysis 63...,"[use, made, technical, slater, m, report, anal...","{outside, pattern, current, contact, organizat...",60,"{'outside': 1, 'pattern': 1, 'current': 1, 'co...","{'outside': 0.013333333333333334, 'pattern': 0..."
2,two kind power an essay bibliographic control ...,"[two, kind, power, an, essay, bibliographic, c...","{obtain, organization, enter, slogan, mankind,...",43,"{'obtain': 1, 'organization': 2, 'enter': 1, '...","{'obtain': 0.014705882352941176, 'organization..."
3,system analysis university final report projec...,"[system, analysis, university, final, report, ...","{remained, lack, service, endorsed, 1960s, aca...",48,"{'remained': 1, 'lack': 1, 'service': 1, 'endo...","{'remained': 0.018867924528301886, 'lack': 0.0..."
4,management game report project brophy p althou...,"[management, game, report, project, brophy, p,...","{complex, introduced, woman, war, professional...",77,"{'complex': 1, 'introduced': 1, 'woman': 1, 'w...","{'complex': 0.010526315789473684, 'introduced'..."
5,abstracting concept method borko h graduate sc...,"[abstracting, concept, method, borko, h, gradu...","{abstractpublishing, preparation, there, autom...",108,"{'abstractpublishing': 1, 'preparation': 2, 't...","{'abstractpublishing': 0.007352941176470588, '..."
6,academic building guide architectural issue so...,"[academic, building, guide, architectural, iss...","{representative, except, architect, college, a...",43,"{'representative': 1, 'except': 1, 'architect'...","{'representative': 0.019230769230769232, 'exce..."
7,academic essay honor guy r lyle farber ei a im...,"[academic, essay, honor, guy, r, lyle, farber,...","{love, individual, honor, admiration, apprenti...",94,"{'love': 1, 'individual': 1, 'honor': 1, 'admi...","{'love': 0.009523809523809525, 'individual': 0..."
8,access college hyman rt study assumed addition...,"[access, college, hyman, rt, study, assumed, a...","{browsing, work, untutored, professional, vari...",79,"{'browsing': 2, 'work': 1, 'untutored': 1, 'pr...","{'browsing': 0.02197802197802198, 'work': 0.01..."
9,access periodical resource palmour ve purpose ...,"[access, periodical, resource, palmour, ve, pu...","{satisfying, service, language, access, bulk, ...",71,"{'satisfying': 1, 'service': 2, 'language': 1,...","{'satisfying': 0.011235955056179775, 'service'..."


In [34]:
qrys = []
for i in range(1,len(qry_set)+1):
    qrys.append(qry_set[str(i)])
    qrdoc=pd.DataFrame(qrys,columns={'Query'})

In [35]:
print(qrdoc)

                                                 Query
0    What problems and concerns are there in making...
1    How can actually pertinent data, as opposed to...
2    What is information science?  Give definitions...
3    Image recognition and any other methods of aut...
4    What special training will ordinary researcher...
..                                                 ...
107  A technique of online instruction and assistan...
108  It is shown that the mapping of a particular a...
109  The "Office of the Future," "Office Technology...
110  An automated document clustering procedure is ...
111  A fast algorithm is described for comparing th...

[112 rows x 1 columns]


In [36]:
qrdoc["text_lower"] = qrdoc["Query"].str.lower()
qrdoc.head()

,Query,text_lower
0,What problems and concerns are there in making...,what problems and concerns are there in making...
1,"How can actually pertinent data, as opposed to...","how can actually pertinent data, as opposed to..."
2,What is information science? Give definitions...,what is information science? give definitions...
3,Image recognition and any other methods of aut...,image recognition and any other methods of aut...
4,What special training will ordinary researcher...,what special training will ordinary researcher...


In [37]:
qrdoc.drop(["text_lower"], axis=1, inplace=True)

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

qrdoc["text_wo_punct"] = qrdoc["Query"].apply(lambda text: remove_punctuation(text))
qrdoc.head()

,Query,text_wo_punct
0,What problems and concerns are there in making...,What problems and concerns are there in making...
1,"How can actually pertinent data, as opposed to...",How can actually pertinent data as opposed to ...
2,What is information science? Give definitions...,What is information science Give definitions ...
3,Image recognition and any other methods of aut...,Image recognition and any other methods of aut...
4,What special training will ordinary researcher...,What special training will ordinary researcher...


In [38]:
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [39]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

qrdoc["text_wo_stop"] = qrdoc["text_wo_punct"].apply(lambda text: remove_stopwords(text))
qrdoc.head()

,Query,text_wo_punct,text_wo_stop
0,What problems and concerns are there in making...,What problems and concerns are there in making...,What problems concerns making descriptive titl...
1,"How can actually pertinent data, as opposed to...",How can actually pertinent data as opposed to ...,How actually pertinent data opposed references...
2,What is information science? Give definitions...,What is information science Give definitions ...,What information science Give definitions poss...
3,Image recognition and any other methods of aut...,Image recognition and any other methods of aut...,Image recognition methods automatically transf...
4,What special training will ordinary researcher...,What special training will ordinary researcher...,What special training ordinary researchers bus...


In [40]:
qrdoc["text_to_lower"] = qrdoc["text_wo_stop"].str.lower()
qrdoc.head(10)

,Query,text_wo_punct,text_wo_stop,text_to_lower
0,What problems and concerns are there in making...,What problems and concerns are there in making...,What problems concerns making descriptive titl...,what problems concerns making descriptive titl...
1,"How can actually pertinent data, as opposed to...",How can actually pertinent data as opposed to ...,How actually pertinent data opposed references...,how actually pertinent data opposed references...
2,What is information science? Give definitions...,What is information science Give definitions ...,What information science Give definitions poss...,what information science give definitions poss...
3,Image recognition and any other methods of aut...,Image recognition and any other methods of aut...,Image recognition methods automatically transf...,image recognition methods automatically transf...
4,What special training will ordinary researcher...,What special training will ordinary researcher...,What special training ordinary researchers bus...,what special training ordinary researchers bus...
5,What possibilities are there for verbal commun...,What possibilities are there for verbal commun...,What possibilities verbal communication comput...,what possibilities verbal communication comput...
6,Describe presently working and planned systems...,Describe presently working and planned systems...,Describe presently working planned systems pub...,describe presently working planned systems pub...
7,Describe information retrieval and indexing in...,Describe information retrieval and indexing in...,Describe information retrieval indexing langua...,describe information retrieval indexing langua...
8,What possibilities are there for automatic gra...,What possibilities are there for automatic gra...,What possibilities automatic grammatical conte...,what possibilities automatic grammatical conte...
9,The use of abstract mathematics in information...,The use of abstract mathematics in information...,The use abstract mathematics information retri...,the use abstract mathematics information retri...


In [41]:
from collections import Counter
cnt = Counter()
for text in qrdoc["text_to_lower"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('retrieval', 101),
 ('the', 97),
 ('information', 91),
 ('systems', 63),
 ('system', 63),
 ('what', 36),
 ('use', 36),
 ('terms', 34),
 ('documents', 34),
 ('data', 32)]

In [42]:
from nltk.stem.porter import PorterStemmer

# Drop the two columns 
#dfdoc.drop(["text_wo_stopfreq"], axis=1, inplace=True) 

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

qrdoc["text_stemmed"] = qrdoc["text_to_lower"].apply(lambda text: stem_words(text))
qrdoc.head()

,Query,text_wo_punct,text_wo_stop,text_to_lower,text_stemmed
0,What problems and concerns are there in making...,What problems and concerns are there in making...,What problems concerns making descriptive titl...,what problems concerns making descriptive titl...,what problem concern make descript titl what d...
1,"How can actually pertinent data, as opposed to...",How can actually pertinent data as opposed to ...,How actually pertinent data opposed references...,how actually pertinent data opposed references...,how actual pertin data oppos refer entir artic...
2,What is information science? Give definitions...,What is information science Give definitions ...,What information science Give definitions poss...,what information science give definitions poss...,what inform scienc give definit possibl
3,Image recognition and any other methods of aut...,Image recognition and any other methods of aut...,Image recognition methods automatically transf...,image recognition methods automatically transf...,imag recognit method automat transform print t...
4,What special training will ordinary researcher...,What special training will ordinary researcher...,What special training ordinary researchers bus...,what special training ordinary researchers bus...,what special train ordinari research businessm...


In [43]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

qrdoc["text_lemmatized"] = qrdoc["text_stemmed"].apply(lambda text: lemmatize_words(text))
qrdoc.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Query,text_wo_punct,text_wo_stop,text_to_lower,text_stemmed,text_lemmatized
0,What problems and concerns are there in making...,What problems and concerns are there in making...,What problems concerns making descriptive titl...,what problems concerns making descriptive titl...,what problem concern make descript titl what d...,what problem concern make descript titl what d...
1,"How can actually pertinent data, as opposed to...",How can actually pertinent data as opposed to ...,How actually pertinent data opposed references...,how actually pertinent data opposed references...,how actual pertin data oppos refer entir artic...,how actual pertin data oppos refer entir artic...
2,What is information science? Give definitions...,What is information science Give definitions ...,What information science Give definitions poss...,what information science give definitions poss...,what inform scienc give definit possibl,what inform scienc give definit possibl
3,Image recognition and any other methods of aut...,Image recognition and any other methods of aut...,Image recognition methods automatically transf...,image recognition methods automatically transf...,imag recognit method automat transform print t...,imag recognit method automat transform print t...
4,What special training will ordinary researcher...,What special training will ordinary researcher...,What special training ordinary researchers bus...,what special training ordinary researchers bus...,what special train ordinari research businessm...,what special train ordinari research businessm...


In [44]:
qrdoc_mesh=pd.DataFrame(qrdoc["text_lemmatized"])

In [45]:
qrdoc_mesh

,text_lemmatized
0,what problem concern make descript titl what d...
1,how actual pertin data oppos refer entir artic...
2,what inform scienc give definit possibl
3,imag recognit method automat transform print t...
4,what special train ordinari research businessm...
...,...
107,a techniqu onlin instruct assist bibliograph d...
108,it shown map particular area scienc case infor...
109,the offic futur offic technolog word process e...
110,an autom document cluster procedur describ req...


In [46]:
def splitbagorwords(text):
  document =text
  bagofwords=document.split(' ')
  return bagofwords

In [47]:
qrdoc_mesh['bagG']=qrdoc_mesh['text_lemmatized'].apply(lambda row: splitbagorwords(row))

In [48]:
qrdoc_mesh

,text_lemmatized,bagG
0,what problem concern make descript titl what d...,"[what, problem, concern, make, descript, titl,..."
1,how actual pertin data oppos refer entir artic...,"[how, actual, pertin, data, oppos, refer, enti..."
2,what inform scienc give definit possibl,"[what, inform, scienc, give, definit, possibl]"
3,imag recognit method automat transform print t...,"[imag, recognit, method, automat, transform, p..."
4,what special train ordinari research businessm...,"[what, special, train, ordinari, research, bus..."
...,...,...
107,a techniqu onlin instruct assist bibliograph d...,"[a, techniqu, onlin, instruct, assist, bibliog..."
108,it shown map particular area scienc case infor...,"[it, shown, map, particular, area, scienc, cas..."
109,the offic futur offic technolog word process e...,"[the, offic, futur, offic, technolog, word, pr..."
110,an autom document cluster procedur describ req...,"[an, autom, document, cluster, procedur, descr..."


In [49]:
def uniq(text1):
  return set(text1)

In [50]:
qrdoc_mesh['uniquewordscc']=qrdoc_mesh['bagG'].apply(lambda row: uniq(row))

In [51]:
qrdoc_mesh

,text_lemmatized,bagG,uniquewordscc
0,what problem concern make descript titl what d...,"[what, problem, concern, make, descript, titl,...","{involv, retriev, problem, relev, usual, descr..."
1,how actual pertin data oppos refer entir artic...,"[how, actual, pertin, data, oppos, refer, enti...","{actual, how, entir, retriev, data, oppos, per..."
2,what inform scienc give definit possibl,"[what, inform, scienc, give, definit, possibl]","{definit, what, possibl, inform, give, scienc}"
3,imag recognit method automat transform print t...,"[imag, recognit, method, automat, transform, p...","{imag, transform, print, automat, method, comp..."
4,what special train ordinari research businessm...,"[what, special, train, ordinari, research, bus...","{train, manag, retriev, special, problem, enco..."
...,...,...,...
107,a techniqu onlin instruct assist bibliograph d...,"[a, techniqu, onlin, instruct, assist, bibliog...","{experiment, techniqu, access, much, describ, ..."
108,it shown map particular area scienc case infor...,"[it, shown, map, particular, area, scienc, cas...","{set, pair, shown, area, author, locat, respec..."
109,the offic futur offic technolog word process e...,"[the, offic, futur, offic, technolog, word, pr...","{howev, current, technolog, tediou, word, grea..."
110,an autom document cluster procedur describ req...,"[an, autom, document, cluster, procedur, descr...","{obtain, set, vocabulari, document, describ, o..."


In [52]:
qrdoc_mesh["UniwordCount"]=qrdoc_mesh['uniquewordscc'].apply(lambda x: len(str(x).split()))

In [53]:
qrdoc_mesh

,text_lemmatized,bagG,uniquewordscc,UniwordCount
0,what problem concern make descript titl what d...,"[what, problem, concern, make, descript, titl,...","{involv, retriev, problem, relev, usual, descr...",15
1,how actual pertin data oppos refer entir artic...,"[how, actual, pertin, data, oppos, refer, enti...","{actual, how, entir, retriev, data, oppos, per...",13
2,what inform scienc give definit possibl,"[what, inform, scienc, give, definit, possibl]","{definit, what, possibl, inform, give, scienc}",6
3,imag recognit method automat transform print t...,"[imag, recognit, method, automat, transform, p...","{imag, transform, print, automat, method, comp...",9
4,what special train ordinari research businessm...,"[what, special, train, ordinari, research, bus...","{train, manag, retriev, special, problem, enco...",17
...,...,...,...,...
107,a techniqu onlin instruct assist bibliograph d...,"[a, techniqu, onlin, instruct, assist, bibliog...","{experiment, techniqu, access, much, describ, ...",49
108,it shown map particular area scienc case infor...,"[it, shown, map, particular, area, scienc, cas...","{set, pair, shown, area, author, locat, respec...",91
109,the offic futur offic technolog word process e...,"[the, offic, futur, offic, technolog, word, pr...","{howev, current, technolog, tediou, word, grea...",70
110,an autom document cluster procedur describ req...,"[an, autom, document, cluster, procedur, descr...","{obtain, set, vocabulari, document, describ, o...",33


In [54]:
def numwords(uniqtext,bagw):
  numOfWords = dict.fromkeys(uniqtext, 0)
  for word in bagw:
    numOfWords[word] += 1
  return numOfWords

In [55]:
qrdoc_mesh["numwords"]=qrdoc_mesh.apply(lambda row:numwords(row['uniquewordscc'],row['bagG']),axis=1)

In [56]:
qrdoc_mesh

,text_lemmatized,bagG,uniquewordscc,UniwordCount,numwords
0,what problem concern make descript titl what d...,"[what, problem, concern, make, descript, titl,...","{involv, retriev, problem, relev, usual, descr...",15,"{'involv': 1, 'retriev': 1, 'problem': 1, 'rel..."
1,how actual pertin data oppos refer entir artic...,"[how, actual, pertin, data, oppos, refer, enti...","{actual, how, entir, retriev, data, oppos, per...",13,"{'actual': 1, 'how': 1, 'entir': 1, 'retriev':..."
2,what inform scienc give definit possibl,"[what, inform, scienc, give, definit, possibl]","{definit, what, possibl, inform, give, scienc}",6,"{'definit': 1, 'what': 1, 'possibl': 1, 'infor..."
3,imag recognit method automat transform print t...,"[imag, recognit, method, automat, transform, p...","{imag, transform, print, automat, method, comp...",9,"{'imag': 1, 'transform': 1, 'print': 1, 'autom..."
4,what special train ordinari research businessm...,"[what, special, train, ordinari, research, bus...","{train, manag, retriev, special, problem, enco...",17,"{'train': 1, 'manag': 1, 'retriev': 1, 'specia..."
...,...,...,...,...,...
107,a techniqu onlin instruct assist bibliograph d...,"[a, techniqu, onlin, instruct, assist, bibliog...","{experiment, techniqu, access, much, describ, ...",49,"{'experiment': 1, 'techniqu': 1, 'access': 1, ..."
108,it shown map particular area scienc case infor...,"[it, shown, map, particular, area, scienc, cas...","{set, pair, shown, area, author, locat, respec...",91,"{'set': 1, 'pair': 1, 'shown': 1, 'area': 4, '..."
109,the offic futur offic technolog word process e...,"[the, offic, futur, offic, technolog, word, pr...","{howev, current, technolog, tediou, word, grea...",70,"{'howev': 1, 'current': 2, 'technolog': 2, 'te..."
110,an autom document cluster procedur describ req...,"[an, autom, document, cluster, procedur, descr...","{obtain, set, vocabulari, document, describ, o...",33,"{'obtain': 2, 'set': 1, 'vocabulari': 1, 'docu..."


In [57]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [58]:
qrdoc_mesh["tf"]=qrdoc_mesh.apply(lambda row:computeTF(row['numwords'],row['bagG']),axis=1)

In [59]:
qrdoc_mesh.head(10)

,text_lemmatized,bagG,uniquewordscc,UniwordCount,numwords,tf
0,what problem concern make descript titl what d...,"[what, problem, concern, make, descript, titl,...","{involv, retriev, problem, relev, usual, descr...",15,"{'involv': 1, 'retriev': 1, 'problem': 1, 'rel...","{'involv': 0.05, 'retriev': 0.05, 'problem': 0..."
1,how actual pertin data oppos refer entir artic...,"[how, actual, pertin, data, oppos, refer, enti...","{actual, how, entir, retriev, data, oppos, per...",13,"{'actual': 1, 'how': 1, 'entir': 1, 'retriev':...","{'actual': 0.07692307692307693, 'how': 0.07692..."
2,what inform scienc give definit possibl,"[what, inform, scienc, give, definit, possibl]","{definit, what, possibl, inform, give, scienc}",6,"{'definit': 1, 'what': 1, 'possibl': 1, 'infor...","{'definit': 0.16666666666666666, 'what': 0.166..."
3,imag recognit method automat transform print t...,"[imag, recognit, method, automat, transform, p...","{imag, transform, print, automat, method, comp...",9,"{'imag': 1, 'transform': 1, 'print': 1, 'autom...","{'imag': 0.1111111111111111, 'transform': 0.11..."
4,what special train ordinari research businessm...,"[what, special, train, ordinari, research, bus...","{train, manag, retriev, special, problem, enco...",17,"{'train': 1, 'manag': 1, 'retriev': 1, 'specia...","{'train': 0.05263157894736842, 'manag': 0.0526..."
5,what possibl verbal commun comput human commun...,"[what, possibl, verbal, commun, comput, human,...","{commun, comput, via, what, possibl, word, spo...",9,"{'commun': 2, 'comput': 1, 'via': 1, 'what': 1...","{'commun': 0.2, 'comput': 0.1, 'via': 0.1, 'wh..."
6,describ present work plan system publish print...,"[describ, present, work, plan, system, publish...","{publish, comput, work, byproduct, code, retri...",18,"{'publish': 1, 'comput': 1, 'work': 1, 'byprod...","{'publish': 0.05555555555555555, 'comput': 0.0..."
7,describ inform retriev index languag what bear...,"[describ, inform, retriev, index, languag, wha...","{retriev, scienc, what, index, inform, describ...",9,"{'retriev': 1, 'scienc': 1, 'what': 1, 'index'...","{'retriev': 0.1111111111111111, 'scienc': 0.11..."
8,what possibl automat grammat contextu analysi ...,"[what, possibl, automat, grammat, contextu, an...","{retriev, grammat, what, possibl, articl, auto...",11,"{'retriev': 1, 'grammat': 1, 'what': 1, 'possi...","{'retriev': 0.09090909090909091, 'grammat': 0...."
9,the use abstract mathemat inform retriev eg gr...,"[the, use, abstract, mathemat, inform, retriev...","{group, retriev, eg, theori, the, mathemat, in...",9,"{'group': 1, 'retriev': 1, 'eg': 1, 'theori': ...","{'group': 0.1111111111111111, 'retriev': 0.111..."


In [60]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [61]:
cv=CountVectorizer()
x=cv.fit_transform(dfdoc_mesh).toarray()

In [62]:
x

array([[0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0]])

In [63]:
def splitbagorwords(text):
  document =text
  bagofwords=document.split(' ')
  return bagofwords

In [64]:
#datasethoc_mesh["CharCount"]=datasethoc_mesh['uniquewordscc'].apply(lambda x: len(x))

In [65]:
dfdoc_mesh.head(10)

,text_lemmatized,bagG,uniquewordscc,UniwordCount,numwords,tf
0,18 edition dewey decimal classification comaro...,"[18, edition, dewey, decimal, classification, ...","{provide, healthy, dewey, continue, 1876, life...",43,"{'provide': 1, 'healthy': 1, 'dewey': 3, 'cont...","{'provide': 0.018867924528301886, 'healthy': 0..."
1,use made technical slater m report analysis 63...,"[use, made, technical, slater, m, report, anal...","{outside, pattern, current, contact, organizat...",60,"{'outside': 1, 'pattern': 1, 'current': 1, 'co...","{'outside': 0.013333333333333334, 'pattern': 0..."
2,two kind power an essay bibliographic control ...,"[two, kind, power, an, essay, bibliographic, c...","{obtain, organization, enter, slogan, mankind,...",43,"{'obtain': 1, 'organization': 2, 'enter': 1, '...","{'obtain': 0.014705882352941176, 'organization..."
3,system analysis university final report projec...,"[system, analysis, university, final, report, ...","{remained, lack, service, endorsed, 1960s, aca...",48,"{'remained': 1, 'lack': 1, 'service': 1, 'endo...","{'remained': 0.018867924528301886, 'lack': 0.0..."
4,management game report project brophy p althou...,"[management, game, report, project, brophy, p,...","{complex, introduced, woman, war, professional...",77,"{'complex': 1, 'introduced': 1, 'woman': 1, 'w...","{'complex': 0.010526315789473684, 'introduced'..."
5,abstracting concept method borko h graduate sc...,"[abstracting, concept, method, borko, h, gradu...","{abstractpublishing, preparation, there, autom...",108,"{'abstractpublishing': 1, 'preparation': 2, 't...","{'abstractpublishing': 0.007352941176470588, '..."
6,academic building guide architectural issue so...,"[academic, building, guide, architectural, iss...","{representative, except, architect, college, a...",43,"{'representative': 1, 'except': 1, 'architect'...","{'representative': 0.019230769230769232, 'exce..."
7,academic essay honor guy r lyle farber ei a im...,"[academic, essay, honor, guy, r, lyle, farber,...","{love, individual, honor, admiration, apprenti...",94,"{'love': 1, 'individual': 1, 'honor': 1, 'admi...","{'love': 0.009523809523809525, 'individual': 0..."
8,access college hyman rt study assumed addition...,"[access, college, hyman, rt, study, assumed, a...","{browsing, work, untutored, professional, vari...",79,"{'browsing': 2, 'work': 1, 'untutored': 1, 'pr...","{'browsing': 0.02197802197802198, 'work': 0.01..."
9,access periodical resource palmour ve purpose ...,"[access, periodical, resource, palmour, ve, pu...","{satisfying, service, language, access, bulk, ...",71,"{'satisfying': 1, 'service': 2, 'language': 1,...","{'satisfying': 0.011235955056179775, 'service'..."


In [66]:
#datasethoc_mesh.to_csv("featurematrixlemm.csv")

In [67]:
#from google.colab import files
#files.download('featurematrixlemm.csv')

In [68]:
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [69]:
docs = dfdoc_mesh['text_lemmatized'].tolist()
tfidfv = TfidfVectorizer(analyzer='word', stop_words='english',)
tfidf_fit = tfidfv.fit(docs)

tfidf_doc = tfidf_fit.transform(docs)
tfidf_doc = pd.DataFrame.sparse.from_spmatrix(tfidf_doc)
tfidf_doc = tfidf_doc.apply(lambda num: num.astype(np.float64))
tfidf_doc.to_csv('/content/drive/MyDrive/NLP/archive/dfdoc_mesh_tfidf.csv')
#content/drive/MyDrive/NLP/archive/

In [70]:
querys = qrdoc_mesh['text_lemmatized'].tolist()
tfidf_qry = tfidf_fit.transform(querys)
tfidf_qry = pd.DataFrame.sparse.from_spmatrix(tfidf_qry)
tfidf_qry = tfidf_qry.apply(lambda num: num.astype(np.float64))
tfidf_qry.to_csv('/content/drive/MyDrive/NLP/archive/qrdoc_mesh_tfidf.csv')

In [71]:
tfidf_doc = pd.read_csv('/content/drive/MyDrive/NLP/archive/dfdoc_mesh_tfidf.csv')
tfidf_doc.drop(tfidf_doc.columns[0], axis = 1, inplace = True)
tfidf_quer = pd.read_csv('/content/drive/MyDrive/NLP/archive/qrdoc_mesh_tfidf.csv')
tfidf_quer.drop(tfidf_quer.columns[0], axis = 1, inplace = True)

In [72]:
tfidf_doc.shape

(1460, 10803)

In [73]:
tfidf_quer.shape

(112, 10803)

In [74]:
from scipy.spatial import distance

In [75]:
def retrieveAll(tfidf_doc, tfidf_qry):
  docT = tfidf_doc.T
  qryT = tfidf_quer.T
  cosine_sim = []
  sorted_ind = []
  for i in qryT.columns:
    result = []
    vec1 = qryT[i]
    for j in docT.columns:
      vec2 = docT[j]
      result.append(1 - distance.cosine(vec1, vec2))
    sort = np.argsort(result)
    sort = sort[::-1]
    sorted_ind.append(sort)
    cosine_sim.append(result)
  return cosine_sim,sorted_ind

In [76]:
cosine_sim, sorted_ind = retrieveAll(tfidf_doc, tfidf_quer)

cols = ['D'+str(i) for i in range(1,len(tfidf_doc)+1)]
cosine_mat = pd.DataFrame(cosine_sim, columns = cols)
cosine_mat.index = np.arange(1, len(cosine_mat) + 1)

sorted_index = pd.DataFrame(sorted_ind, columns = np.arange(1, len(tfidf_doc) + 1))
sorted_index.index = np.arange(1, len(sorted_index) + 1)

cosine_mat.to_csv('/content/drive/MyDrive/NLP/archive/doc_sw_cosine.csv')
sorted_index.to_csv('/content/drive/MyDrive/NLP/archive/doc_sw_sorted_index.csv')

/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [77]:
cos_matrix = pd.read_csv('/content/drive/MyDrive/NLP/archive/doc_sw_cosine.csv')
cos_matrix.drop(cos_matrix.columns[0], axis = 1, inplace = True)
sort_ind_value = pd.read_csv('/content/drive/MyDrive/NLP/archive/doc_sw_sorted_index.csv')
sort_ind_value.drop(sort_ind_value.columns[0], axis = 1, inplace = True)

cosine_sim_value = cos_matrix.values.tolist()
sorted_index_value = sort_ind_value.values.tolist()

In [78]:
relation = []
for i in range(1,len(qry_set)+1):
  if i in rel_set.keys():
    relation.append(rel_set[i])
  else:
    relation.append(np.nan)

In [79]:
def valid_ret(cos, sort_ind):
  valid = [int(ind) for ind in sort_ind if cos[ind] > 0]
  return valid

qrylist = ['Q'+str(i) for i in range(1,len(tfidf_quer)+1)] # if i in rel_set.keys()]
Result = pd.DataFrame(index = qrylist) 
Result['Cosine Distance Values'] = cosine_sim_value
Result['Sorted Doc Index'] = sorted_index_value 
#Result["Unique Doc Count"]=Result['Sorted Doc Index'].apply(lambda x: len(str(x).split()))
Result['Positive Retrieved'] = Result.apply(lambda row: valid_ret(row['Cosine Distance Values'], row['Sorted Doc Index']), axis = 1)


In [80]:

Result.head(20)

,Cosine Distance Values,Sorted Doc Index,Positive Retrieved
Q1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0506653386104...","[1029, 1444, 872, 1315, 223, 77, 653, 92, 243,...","[1029, 1444, 872, 1315, 223, 77, 653, 92, 243,..."
Q2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1154, 57, 561, 1157, 1362, 1125, 393, 769, 30...","[1154, 57, 561, 1157, 1362, 1125, 393, 769, 30..."
Q3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[769, 240, 1459, 489, 481, 482, 483, 484, 485,...","[769, 240]"
Q4,"[0.0, 0.0, 0.047002943403714, 0.01823544291215...","[1223, 1208, 564, 526, 1279, 178, 1421, 1408, ...","[1223, 1208, 564, 526, 1279, 178, 1421, 1408, ..."
Q5,"[0.0, 0.0337168731521788, 0.0, 0.0114521242372...","[240, 769, 861, 143, 201, 933, 586, 1353, 482,...","[240, 769, 861, 143, 201, 933, 586, 1353, 482,..."
Q6,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[665, 648, 42, 328, 65, 576, 1090, 642, 78, 48...","[665, 648, 42, 328, 65, 576, 1090, 642, 78, 48..."
Q7,"[0.0233345579550154, 0.0410473560936289, 0.031...","[262, 667, 747, 1348, 283, 734, 1013, 36, 1236...","[262, 667, 747, 1348, 283, 734, 1013, 36, 1236..."
Q8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[52, 85, 43, 193, 687, 437, 769, 471, 714, 780...","[52, 85, 43, 193, 687, 437, 769, 471, 714, 780..."
Q9,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[769, 240, 1459, 489, 481, 482, 483, 484, 485,...","[769, 240]"
Q10,"[0.0, 0.0732073346775115, 0.0, 0.0, 0.02192627...","[656, 769, 1280, 570, 671, 252, 649, 1403, 113...","[656, 769, 1280, 570, 671, 252, 649, 1403, 113..."


In [81]:
print(type((Result['Cosine Distance Values'].iloc[0])))
print(type((Result['Sorted Doc Index'].iloc[0])[0]))
print(type((Result['Positive Retrieved'].iloc[0])[0]))
print(type((Result['Cosine Distance Values'].iloc[0])[0]))

<class 'list'>
<class 'int'>
<class 'int'>
<class 'float'>


In [82]:
def topMatch(sorted_index, rel_index, top):
    ret_index = sorted_index[0:top]
    rel_index = [l-1 for l in rel_index]
    rel_mtch = []
    for index in rel_index:
        if index in ret_index:
            rel_mtch.append(index)
    return rel_mtch

In [83]:
def ret_rel(rel,ret):
  return (len(ret), len(rel))

# precision = relevant retrieved/total retrieved
def precision(ret_rel, n):
  ret, rel = ret_rel
  return ret/n

# recall =  relevent retrieved/total relevant
def recall(ret_rel,n):
  ret, rel = ret_rel
  return ret/rel

In [84]:
total_documents = len(dfdoc_mesh["tf"])

In [85]:
total_documents

1460

In [86]:
def inverse_doc_freq(word):
    try:
        word_occurance = dfdoc_mesh[word] + 1
    except:
        word_occurance = 1
    return np.log(total_documents/word_occurance)

In [87]:
#idfDict